# HW03: Supervised Machine Learning, Regression and XGBoost

Remember that these homework work as a completion grade. **You can skip one section without losing credit.**

## Load and Pre-process Text
We do sentiment analysis on the [Movie Review Data](https://www.cs.cornell.edu/people/pabo/movie-review-data/). If you would like to know more about the data, have a look at [the paper](https://www.cs.cornell.edu/home/llee/papers/pang-lee-stars.pdf) (but no need to do so).

In [1]:
# In this tutorial, we do sentiment analysis
# download the data
#!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar xf aclImdb_v1.tar.gz

!wget https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
!wget https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_whole_review.tar.gz
 
!tar xf scale_data.tar.gz 
!tar xf scale_whole_review.tar.gz

--2021-03-17 10:44:15--  https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4029756 (3,8M) [application/x-gzip]
Saving to: ‘scale_data.tar.gz’

scale_data.tar.gz   100%[===================>]   3,84M  2,94MB/s    in 1,3s    

2021-03-17 10:44:17 (2,94 MB/s) - ‘scale_data.tar.gz’ saved [4029756/4029756]

--2021-03-17 10:44:18--  https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_whole_review.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8853204 (8,4M) [application/x-gzip]
Saving to: ‘scale_whole_review.tar.gz’

scale_whole_review. 100%[===================>]   8,44M  2,71MB/s  

First, we have to load the data for which we provide the function below. Note how we also preprocess the text using gensim's simple_preprocess() function and how we already split the data into a train and test split.

In [1]:
import os
from gensim.utils import simple_preprocess
def load_data():
    examples, labels = [], []
    authors = os.listdir("scale_whole_review")
    for author in authors:
        path = os.listdir(os.path.join("scale_whole_review", author, "txt.parag"))
        fn_ids = os.path.join("scaledata", author, "id." + author)
        fn_ratings = os.path.join("scaledata", author, "rating." + author)
        with open(fn_ids) as ids, open(fn_ratings) as ratings:
            for idx, rating in zip(ids, ratings):
                labels.append(float(rating.strip()))
                filename_text = os.path.join("scale_whole_review", author, "txt.parag", idx.strip() + ".txt")
                with open(filename_text, encoding='latin-1') as f:
                    examples.append(" ".join(simple_preprocess(f.read())))
    return examples, labels
                  
X,y  = load_data()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print ("text:", X_train[0], "\nlabel:", y_train[0])

text: bloody child the director writer cinematographer nina menkes screenwriter tinka menkes editors nina and tina menkes cast tinka menkes captain sherry sibley murdered wife robert mueller murderer russ little sergeant jack hara enlisted man runtime mirage reviewed by dennis schwartz an amazingly strange film confusing and not thoroughly enjoyable but film found more interesting than thought possible at first viewing this experimental film in minimalist story telling film consisting of disturbing visualizations and almost no dialogue had concept that was greater than how the film turned out it felt at times like was watching paint dry on the wall but the reward for sitting through those excruciatingly redundant scenes was in seeing something different something that cast spell of sorcery over terrible incident as believe the film in its unique and sometimes shrill voice does justice in commenting on the violence in american society especially against women the film uses its impressio

## Vectorize the data

In [2]:
# train a TF_IDF Vectorizer on X_train and vectorize X_train and X_test
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(min_df=0.01, # at min 1% of docs
                        max_df=.5,  
                        stop_words='english',
                        ngram_range=(1,2))

##TODO train vectorizer
vec.fit(X_train)
##TODO transform X_train to TF-IDF values
X_train_tfidf = vec.transform(X_train)
##TODO transform X_test to TF-IDF values
X_test_tfidf = vec.transform(X_test)

In [3]:
##TODO scale both training and test data with the standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
scaler.fit(X_train_tfidf)
X_train_scaled = scaler.transform(X_train_tfidf)
X_test_scaled = scaler.transform(X_test_tfidf)

## ElasticNet

In [4]:
##TODO train an elastic net on the transformed output of the scaler
from sklearn.linear_model import ElasticNet

en = ElasticNet(alpha=0.01)

##TODO train the ElasticNet
en.fit(X_train_scaled, y_train)
##TODO predict the testset
pred = en.predict(X_test_scaled)

In [5]:
print(pred)
print(y_test)
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error, balanced_accuracy_score
##TODO print mean squared error and r2 score on the test set
print(f'Mean squared error: {mean_squared_error(y_test, pred)}')
print(f'R2 score: {r2_score(y_test, pred)}')

[0.53101505 0.64887074 0.71933284 ... 0.53997695 0.49289631 0.54983887]
[0.5, 0.7, 0.83, 0.3, 0.3, 0.79, 0.64, 0.81, 0.6, 0.3, 0.7, 0.2, 0.8, 0.75, 0.6, 0.4, 0.8, 0.3, 0.7, 0.4, 0.9, 0.4, 0.5, 0.6, 0.5, 0.5, 0.47, 0.5, 0.3, 0.8, 0.5, 0.6, 0.3, 0.4, 0.4, 0.2, 0.6, 0.7, 0.7, 0.4, 0.5, 0.2, 0.7, 0.6, 0.8, 0.4, 0.3, 0.6, 0.61, 0.5, 0.7, 0.2, 0.8, 0.6, 0.3, 0.4, 0.7, 0.5, 0.6, 0.7, 0.7, 0.78, 0.5, 0.4, 0.3, 0.3, 0.7, 0.5, 0.9, 0.6, 0.2, 0.49, 0.8, 0.8, 0.6, 0.6, 0.5, 0.46, 0.75, 0.3, 0.6, 0.7, 0.5, 0.4, 0.6, 0.3, 0.6, 0.5, 0.6, 0.4, 0.4, 0.7, 0.6, 0.7, 0.6, 0.4, 0.5, 0.75, 0.3, 0.7, 0.6, 0.8, 0.7, 0.6, 0.4, 0.35, 0.7, 0.5, 0.92, 0.5, 0.3, 0.5, 0.3, 0.5, 0.44, 0.5, 0.65, 0.6, 0.4, 0.8, 0.8, 0.71, 0.7, 0.4, 0.6, 0.7, 1.0, 0.6, 0.9, 0.7, 0.7, 0.9, 0.8, 0.6, 0.7, 0.6, 0.6, 0.8, 0.3, 0.8, 0.58, 0.8, 0.56, 0.8, 0.2, 0.59, 0.6, 0.6, 0.8, 0.7, 0.5, 0.4, 0.8, 0.6, 0.5, 0.5, 0.55, 0.8, 0.84, 0.7, 0.5, 0.7, 0.3, 0.1, 0.9, 0.7, 0.8, 0.4, 0.7, 0.79, 0.5, 0.8, 0.6, 0.4, 0.5, 0.6, 0.9, 0.6, 0.6, 0.7, 0.5,

## Logistic Regression

Next, we train an OLS model doing binary prediction on these movie reviews. Two get two bins, we transform the continuous ratings into two classes, where one class contains all the negative ratings (value < 0.5), the other class all the positive ratings (value > 0.5)

In [6]:
y_train = [1 if i >= 0.5 else 0 for i in y_train]
y_test = [1 if i >= 0.5 else 0 for i in y_test]


In [7]:
##TODO train logistic regression on X_train
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression()

##TODO train a logistic regression
logistic_regression.fit(X_train_scaled, y_train)
##TODO predict the testset 
pred = logistic_regression.predict(X_test_scaled)
##since we have continuous output, we need to post-process our labels into two classes. We choose a threshold of 0.5 
def map_predictions(predicted):
    predicted = [1 if i > 0.5 else 0 for i in predicted]
    return predicted

##TODO print the accuracy of our classifier on the testset
pred = map_predictions(pred)
print(f'Accuracy: {accuracy_score(y_test, pred)}')

Accuracy: 0.8075060532687651


/Users/Hendrik/opt/anaconda3/envs/legal_dna/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [14]:
import pandas as pd
## TODO print the 10 most informative words of the regression (the 10 words having the highest coefficients)
print (pd.Series(index=vec.get_feature_names(), data=logistic_regression.coef_.flatten()).nlargest(10))

great            0.229969
effective        0.210105
surprisingly     0.201393
punches          0.197310
success          0.185941
fascinating      0.184809
influenced       0.182900
investigating    0.179927
brilliant        0.179231
best             0.173113
dtype: float64


## XGBoost

Lastly, we train an XGBoost classifier to do topic prediction on the AG news dataset, which is a multi-class prediction problem (4 classes). We again have to vectorize the data, train the classifier, predict the testset and output an evaluation metric (we go for accuracy).

In [19]:
!pip install xgboost

     |████████████████████████████████| 1.2 MB 2.1 MB/s eta 0:00:01


In [20]:
#Import the AG news dataset (same as hw01)
#Download them from here 
# !wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
df["text"] = df["title"] + " " + df["lead"]
df.head()

--2021-03-17 11:18:39--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  28,10M  1,78MB/s    in 15s     

2021-03-17 11:18:57 (1,86 MB/s) - ‘train.csv’ saved [29470338/29470338]



,label,title,lead,text
0,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
1,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
2,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
3,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."
4,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...,"Stocks End Up, But Near Year Lows (Reuters) Re..."


In [21]:
# vectorize the data
from sklearn.feature_extraction.text import TfidfVectorizer

# only consider 10% of the data
dfs = df.sample(frac=0.1)

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(dfs["text"], dfs["label"], test_size=0.33, random_state=42)

vec = TfidfVectorizer(min_df=5, # at min 1% of docs
                        max_df=.5,  
                        stop_words='english',
                        max_features=2000,
                        ngram_range=(1,2))

# transform into TF-IDF values
X_train_tfidf = vec.fit_transform(X_train).todense()
X_test_tfidf = vec.transform(X_test).todense()


XGBoost provides an interface to SKLearn classifiers, e.g. they implement the same train and predict methods as an SKLearn classifier would. If you are interested in a more detailed overview, have a look at the [official documentation](https://xgboost.readthedocs.io/en/latest/python/index.html).

In [25]:
param_dist = {'objective':'multi:softmax', 'num_class': 5, 'n_estimators':25}
# note how we only have 4 labels, but we need to pass "num_class": 5
# if we pass "num_class": 4, we get the error "label must be in [0, num_class)."
import xgboost as xgb

clf = xgb.XGBModel(**param_dist)

##TODO train the XGBModel 
clf.fit(X_train_tfidf, y_train)
##TODO predict the testset 
pred = clf.predict(X_test_tfidf)
##TODO evaluate the predictions using accuracy as a metric
print(f'Accuracy: {accuracy_score(y_test, pred)}')

[11:35:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.8078282828282828


With scaling no change in score:

In [26]:
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_tfidf)
X_test_scaled = scaler.transform(X_test_tfidf)
clf_ = xgb.XGBModel(**param_dist)

##TODO train the XGBModel
clf_.fit(X_train_scaled, y_train)
##TODO predict the testset
pred = clf_.predict(X_test_scaled)
print(f'Accuracy: {accuracy_score(y_test, pred)}')

[11:37:03] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.8078282828282828
